## This notebook is an example of how to pipeline two models. 
A video stream from a local camera is processed by the face detection model. The face detection results are then processed by the mask detection model, one face bounding box at a time. Combined result is then displayed.

This example uses `mystreams` streaming toolkit.

**Access to camera is required to run this sample.**

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below.

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `camera_id` or in .env file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

### Specify where do you want to run your inferences and camera index here

In [ ]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment
camera_id = 0         # camera index or URL; 0 to use default local camera, None to take from .env file

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools, cv2
from mystreams import *

In [ ]:
# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")
mask_det_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1")

# adjust mask model properties
mask_det_model.output_top_k = 1 # report single result with top confidence
mask_det_model.image_backend = 'opencv' 
mask_det_model.input_numpy_colorspace = 'BGR'
mask_det_model._model_parameters.InputImgFmt = ['JPEG']

# adjust face model properties
face_det_model.overlay_show_probabilities = True
face_det_model.output_confidence_threshold = 0.7
face_det_model.image_backend = 'opencv'
face_det_model._model_parameters.InputImgFmt = ['JPEG']

In [ ]:
# Define mask detection gizmo (in mystreams terminology)
class MaskDetectionGizmo(AiGizmoBase):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._cur_result = None
        
    def on_result(self, result):
        
        # here result.info contains StreamData object used for AI inference (because AiGizmoBase does it this way);
        # and result.info.meta contains metainfo dictionary placed by AiObjectDetectionCroppingGizmo, 
        # because in our pipeline it is connected as a source of this gizmo
        meta = result.info.meta
        if "original_result" in meta: # new result
            if self._cur_result is not None:
                # send previous result
                self.send_result(StreamData(self._cur_result.image, self._cur_result))                
            self._cur_result = meta["original_result"]
        
        if "cropped_index" in meta:
            # apply mask presence/absence label
            if len(result.results) > 0:
                self._cur_result.results[meta["cropped_index"]]["label"] = result.results[0]["label"]
                self._cur_result.results[meta["cropped_index"]]["score"] = result.results[0]["score"]

In [ ]:
# create composition object
c = Composition();

# create gizmos adding them to composition
source = c.add(VideoSourceGizmo(camera_id)) # video source
face_detection = c.add(AiObjectDetectionCroppingGizmo(face_det_model)) # face detection gizmo, which outputs cropped images for all detected faces
mask_detection = c.add(MaskDetectionGizmo(mask_det_model)) # mask detection AI model
face_display = c.add(VideoDisplayGizmo("Faces", show_ai_overlay=True, show_fps=True)) # display

# connect gizmos to create pipeline
source >> face_detection >> mask_detection >> face_display

# start execution of composition 
c.start()